In [ ]:
%reload_ext autoreload
%autoreload 2

# from drift.ftx import ftx_trade_to_target_position, recent_ftx_trades
from drift.drift import Drift, load_config, MARKET_INDEX_TO_PERP
import pandas as pd
import os
pd.options.plotting.backend = "plotly"

In [ ]:
USER_AUTHORITY = load_config() # TODO: replace with user wallet public key
USER_AUTHORITY

In [ ]:
from anchorpy import Idl, Program, Provider

In [ ]:
drift = Drift(USER_AUTHORITY)
await drift.load()
history_df = await drift.load_history_df()
history_df.keys()

In [ ]:
drift.market_summary()

# trades

In [ ]:
trdf = history_df['trade'].copy().sort_index()
trdf['user_authority'] = trdf['user_authority'].astype(str)

duration = (trdf['fee'].index[-1] - trdf['fee'].index[0])
duration = duration.seconds/60/60

for x in ['fee','quote_asset_amount']:
    trdf[x] /= 1e6
    trdf[x] = trdf[x].round(2)
for x in ['base_asset_amount']:
    trdf[x] /= 1e13
for x in ['mark_price_after','mark_price_before','oracle_price']:
    trdf[x] /= 1e10
    
# show volume of traders in 1024 most recent trades
toshow = trdf.groupby(['user_authority', 'market_index'])\
[['base_asset_amount','quote_asset_amount','fee']].sum().sort_values('fee', ascending=False)
print(int(duration*100)/100, 'hours')
# calculate interpolated daily fee spend
toshow['hourly_avg_fee'] = (toshow['fee']/duration ).round(2)
toshow

# liquidations

In [ ]:
liqs = history_df['liquidation']
liqs = liqs[(liqs['margin_ratio']!=0)]
liqs.columns
# (liqs['feeToLiquidator']/1e6).resample('D').sum().plot(kind='bar')
for col in ['total_collateral', 'collateral','unrealized_pnl','fee_to_insurance_fund','fee_to_liquidator',
            'liquidation_fee','base_asset_value','base_asset_value_closed']:
    liqs[col] /= 1e6
liqs['liquidator'] = liqs['liquidator'].astype(str)    
    
# liqs.sort_values('feeToLiquidator').to_csv("~/drift_liquidations_20211119.csv")

(liqs.loc['2021-11-18':])#['baseAssetValueClosed'])#.resample('1MIN').sum().plot()

((liqs.loc['2021-11-18'])['margin_ratio']/1e3).hist()

(liqs['fee_to_insurance_fund'].cumsum()).plot()

In [ ]:
liqs.groupby(['liquidator', 'partial'])['fee_to_liquidator'].agg(['sum','mean','median','count'])

# curve
adjustments to k and repeg events

In [ ]:
curvedffull = history_df['curve'].copy().sort_index()
for marketIndex in curvedffull.market_index.unique():
    curvedf = curvedffull[curvedffull.market_index == marketIndex]
    print(curvedf.columns)
    cdf = curvedf[['total_fee', 'total_fee_minus_distributions','adjustment_cost']]/1e6
    fig = cdf.plot(title=MARKET_INDEX_TO_PERP[marketIndex])
    fig.show()

# deposits

In [ ]:
deposits = history_df['deposit'].loc['2021':]
d = deposits.direction[0]
assert('deposit' in str(d).lower())
deposit_ts = (deposits.apply(lambda x: x['amount']*-1 if x['direction']!=d else x['amount'],axis=1)/1e6)
deposit_ts.sort_index().cumsum().plot()

In [ ]:
tr = history_df['trade'].sort_index()
tr[tr.user_authority.astype(str)!=USER_AUTHORITY].sort_index()
usrcheck = tr[tr.user_authority.astype(str)==USER_AUTHORITY]
d = usrcheck.direction.values[0]
pos_ts = (usrcheck.apply(lambda x: x['base_asset_amount']*-1 
                         if x['direction']!=d else x['base_asset_amount'],axis=1)/1e13)
pos_ts.cumsum().plot()

# funding rate

In [ ]:
import numpy as np
frfull = history_df['fundingRate'].sort_index()

In [ ]:
for marketIndex in frfull.market_index.unique():
    fr = frfull[frfull['market_index']==marketIndex]  
    (fr[['oracle_price_twap','mark_price_twap']]/1e10).replace(0,np.nan).dropna().plot()
    fr_hand = (fr[['oracle_price_twap','mark_price_twap']].diff(axis=1)/1e10).iloc[:,-1]\
    .replace(0,np.nan)/24
    fr_prot = (fr[['cumulative_funding_rate_long', 'cumulative_funding_rate_short']]/1e14)\
    .replace(0,np.nan).diff()
    dfplt = pd.concat([fr_hand, fr_prot],axis=1)
    dfplt = (dfplt*100).mul(1/(fr['oracle_price_twap']/1e10), axis=0).dropna().tail(7*24)
    
    dfplt = dfplt.rename({'cumulative_funding_rate_long':'long_funding_rate',
                  'cumulative_funding_rate_short':'short_funding_rate',
                  'mark_price_twap':'balanced_funding'
                 },axis=1)
    fig = dfplt.plot(title=MARKET_INDEX_TO_PERP[marketIndex]+' funding rate %')
    fig.show()

In [ ]:
# ?dfplt.plot

In [ ]:
mkt0 = drift.mkt_account.markets[0]
arbJ_position = drift.bot_position.positions[0].baseAssetAmount/1e13
total_position = (mkt0.baseAssetAmountLong - mkt0.baseAssetAmountShort)/1e13
arbJ_position/total_position

In [ ]:
arbJ_position = drift.bot_position.positions[0].baseAssetAmount/1e13
print(drift.base_asset_imbalance() - arbJ_position)
drift.base_asset_imbalance(), arbJ_position

In [ ]:
drift.all_users[0]

In [ ]:
user_summary_df = drift.user_summary()
user_summary_df['realized_pnl'].sum()

In [ ]:
user_summary_df[user_summary_df.Address == ('4CLaLrTr7juGATt6FAaw8JksSNGdTEGaA78otYJbdHDs')]

In [ ]:
from drift.clearinghouse import ClearingHouseUser

In [ ]:
user = ClearingHouseUser(drift, USER_AUTHORITY)
await user.positions()